## Sacar las features de una lista de canciones desde una playlist

Proceso:

- Investigar cómo sacar features en serie con ID de canciones con API de Spotify
- Descargar features de las listas de top200
- Juntar Dataframes de features con el de top200 

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# Instalo librería Spotipy y poner credenciales
# !pip install spotipy

In [2]:
#Preparación de Spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Código para sacar listado de más de 100 canciones(spotify no te deja más de lotes de 100)) de playlist

def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

tracks = get_playlist_tracks('akaki', '1ebwiiN18Gmhfj6e24pT3X')
len(tracks)

437

In [7]:
# Código para sacar id de todas las canciones de dicho listado anterior

ids = []
# results = sp.user_playlist_tracks('akaki', '1ebwiiN18Gmhfj6e24pT3X')
for code in tracks:
    track = code['track']
    ids.append(track['id'])
len(ids)

437

In [ ]:
# Tuve que quitar las Nonetypes para que no me diera error

ids_clean = [x for x in ids if x != None]
len(clean)

435

In [ ]:
#Para obtener código de canción se accede a lista tracks que dentro tiene un diccionario dentro de otro diccionario y así, y entramos en profundidad hasta el id.
# song1_code = tracks[0]['track']['id']

# Probando...
'''
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('125154244', '5CGooYaNL93cVrzmkyhAyN')
'''

"\ndef getTrackIDs(user, playlist_id):\n    ids = []\n    playlist = sp.user_playlist(user, playlist_id)\n    for item in playlist['tracks']['items']:\n        track = item['track']\n        ids.append(track['id'])\n    return ids\n\nids = getTrackIDs('125154244', '5CGooYaNL93cVrzmkyhAyN')\n"

In [ ]:
# Función para sacar la info de determinadas ID de canciones
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  valence = features[0]['valence']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  id = features[0]['id']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, valence, tempo, time_signature, id]
  return track

In [ ]:
# Loop para sacar la info de todas las canciones de la lista anterior
#OJO, TARDA BASTANTE!!

tracks = []
for i in range(len(ids_clean)):
    time.sleep(.5)
    track = getTrackFeatures(ids_clean[i])
    tracks.append(track)

In [ ]:
# Meterlo en Dataframe y exportar a CSV
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness',
                                     'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence',
                                     'tempo', 'time_signature', 'id'])
# df.to_csv("spotify.csv", sep = ',')



ValueError: 18 columns passed, passed data had 16 columns

In [ ]:
print(df.shape)
df.head()

(435, 16)

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Pride,Nip/Tuck,Various Artists,2011-08-09,377280,0,0.290,0.32200,0.290,0.906,0.0091,0.7110,-7.242,0.2540,170.014,4
1,Mars,Levi's Unfamous Music Awards,Various Artists,2010-04-01,317492,0,0.430,0.00722,0.430,0.764,0.0040,0.0982,-6.556,0.0416,149.776,4
2,"Oh, No",I Oughtta Give You A Shot In The Head For Maki...,Shivaree,1999-01-01,204573,12,0.742,0.06360,0.742,0.533,0.8620,0.0739,-6.440,0.0274,96.366,4
3,Pressure,Pressure,Skindred,2004,211146,17,0.459,0.05910,0.459,0.953,0.0000,0.4500,-4.387,0.2170,191.639,4
4,Chasing Pirates,The Fall,Norah Jones,2009-11-17,160906,56,0.850,0.54700,0.850,0.468,0.0667,0.1060,-9.090,0.0390,106.974,4


SyntaxError: invalid syntax (<ipython-input-5-d7c19b7242c0>, line 1)

In [ ]:
#Otra forma:

def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

dataframe = analyze_playlist("akaki", "1ebwiiN18Gmhfj6e24pT3X")

print(dataframe.shape)
dataframe.head()